# GitHub API Analyst Homework - Patricia M.

In [ ]:
import requests
import json
import time
import os
import pandas as pd
from getpass import getpass

# Solicita el token de GitHub de forma segura


In [ ]:
github_token = getpass("🔐 Ingresa tu token personal de GitHub: ")
os.environ["GITHUB_TOKEN"] = github_token


🔐 Ingresa tu token personal de GitHub: ··········


# Obtiene el token desde variable de entorno

In [ ]:
TOKEN = os.getenv("GITHUB_TOKEN")

# Encabezados requeridos para la autenticación y uso correcto de la API

In [ ]:
headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28"
}

In [ ]:
# Define tu token de autenticación
GITHUB_TOKEN = "tu_token_personal_aquí"  # 👈🏽 Reemplaza con tu token de GitHub




In [ ]:
# Configura los headers para autenticarte en la API de GitHub
headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28"
}
headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28"
}

def get_repositories(query, per_page=10):
    url = f"https://api.github.com/search/repositories?q={query}&per_page={per_page}"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def get_commits(owner, repo):
    url = f"https://api.github.com/repos/{owner}/{repo}/commits"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()


# Función para realizar solicitudes HTTP y manejar errores comunes

In [ ]:
def hacer_peticion(url):
    while True:
        respuesta = requests.get(url, headers=headers)
        if respuesta.status_code == 200:
            return respuesta.json()
        elif respuesta.status_code == 403 and 'X-RateLimit-Remaining' in respuesta.headers:
            tiempo_reinicio = int(respuesta.headers.get("X-RateLimit-Reset", time.time() + 60))
            esperar = max(tiempo_reinicio - int(time.time()), 1)
            print(f"Límite alcanzado. Esperando {esperar} segundos...")
            time.sleep(esperar)
        elif respuesta.status_code == 401:
            print("Error 401: Token inválido o sin permisos.")
            return []
        else:
            print(f"Error {respuesta.status_code}: {respuesta.text}")
            return []


In [ ]:
# Función para recorrer múltiples páginas de resultados

def obtener_paginas(url_base, por_pagina=30, paginas_max=3):
    resultados = []
    for pagina in range(1, paginas_max + 1):
        url = f"{url_base}&per_page={por_pagina}&page={pagina}"
        datos = hacer_peticion(url)
        if not datos:
            break
        if isinstance(datos, list):
            resultados.extend(datos)
        else:
            resultados.extend(datos.get("items", []))
    return resultados

In [ ]:
# Buscar repositorios con la palabra clave "data science"
print("\n--- Repositorios sobre 'data science' ---")
busqueda_url = "https://api.github.com/search/repositories?q=data+science&sort=stars"
repositorios = obtener_paginas(busqueda_url, por_pagina=5, paginas_max=2)
for repo in repositorios:
    print(f"- {repo['full_name']} (⭐ {repo['stargazers_count']})")

# Obtener commits de un repositorio público
print("\n--- Commits de pandas-dev/pandas ---")
url_commits = "https://api.github.com/repos/pandas-dev/pandas/commits?"
commits = obtener_paginas(url_commits, por_pagina=5, paginas_max=2)
for commit in commits[:5]:
    mensaje = commit['commit']['message']
    sha = commit['sha'][:7]
    print(f"- {mensaje} ({sha})")

# Consultar el contenido del repositorio en su raíz
print("\n--- Archivos en la raíz de pandas-dev/pandas ---")
url_contenido = "https://api.github.com/repos/pandas-dev/pandas/contents/"
contenido = hacer_peticion(url_contenido)
for item in contenido[:5]:
    print(f"- {item['name']} ({item['type']})")

# Simulación de error con token incorrecto
print("\n--- Prueba de error con token inválido ---")
def prueba_error():
    headers_erroneos = headers.copy()
    headers_erroneos["Authorization"] = "Bearer token_invalido"
    url_error = "https://api.github.com/user"
    respuesta = requests.get(url_error, headers=headers_erroneos)
    if respuesta.status_code == 401:
        print("Error 401: No autorizado (token inválido)")
    elif respuesta.status_code == 403:
        print("Error 403: Límite de peticiones alcanzado")
    else:
        print(f"Código inesperado: {respuesta.status_code}")

prueba_error()


--- Repositorios sobre 'data science' ---
Límite alcanzado. Esperando 7 segundos...
Límite alcanzado. Esperando 1 segundos...
- jakevdp/PythonDataScienceHandbook (⭐ 44939)
- microsoft/Data-Science-For-Beginners (⭐ 29922)
- donnemartin/data-science-ipython-notebooks (⭐ 28338)
- eugeneyan/applied-ml (⭐ 28095)
- academic/awesome-datascience (⭐ 26843)
- datasciencemasters/go (⭐ 25518)
- ossu/data-science (⭐ 19981)
- FavioVazquez/ds-cheatsheets (⭐ 15518)
- virgili0/Virgilio (⭐ 14125)
- tangyudi/Ai-Learn (⭐ 11420)

--- Commits de pandas-dev/pandas ---
- BUG: Fix Index.equals between object and string (#61541) (b876c67)
- TST[string]: update expecteds for using_string_dtype to fix xfails (#61727)

* TST: update expecteds for using_string_dtype to fix xfails

* Update to_dict_of_blocks test to hardcode object dtype

* Comment

* Split test, update expected, targeted xfails

* Update json test

* revert commented-out (e635c3e)
- CI: Remove PyPy references in CI testing (#61814) (f94b430)
- fea

In [ ]:
# Función para exportar los datos de los repositorios a CSV
def exportar_repositorios_csv(repositorios, archivo_salida="repositorios.csv"):
    datos = []
    for repo in repositorios:
        datos.append({
            "repo_name": repo.get("name"),
            "repo_full_name": repo.get("full_name"),
            "owner": repo.get("owner", {}).get("login"),
            "stars": repo.get("stargazers_count"),
            "forks": repo.get("forks_count"),
            "created_at": repo.get("created_at"),
            "updated_at": repo.get("updated_at"),
            "url": repo.get("html_url"),
            "description": repo.get("description")
        })
    df = pd.DataFrame(datos)
    df.to_csv(archivo_salida, index=False)
    print(f"✅ Repositorios exportados en: {archivo_salida}")

In [ ]:
# Función para exportar los datos de los commits a CSV
def exportar_commits_csv(commits, archivo_salida="commits.csv"):
    datos = []
    for c in commits:
        datos.append({
            "commit_sha": c.get("sha", "")[:7],
            "commit_message": c.get("commit", {}).get("message"),
            "commit_author": c.get("commit", {}).get("author", {}).get("name")
        })
    df = pd.DataFrame(datos)
    df.to_csv(archivo_salida, index=False)
    print(f"✅ Commits exportados en: {archivo_salida}")

In [ ]:
# Exportar como extra de la tarea
exportar_repositorios_csv(repositorios)
exportar_commits_csv(commits)


✅ Repositorios exportados en: repositorios.csv
✅ Commits exportados en: commits.csv
